# Introducción a TensorFlow
El primer paso para escribir programas utilizando TensorFlow es importar el módulo. Por convención, se importa al mismo en la variable `tf`.

In [1]:
import tensorflow as tf

Se puede considerar que todo programa escrito con TensorFlow consta de dos partes:
1. Construcción de la **gráfica computacional** ([tf.Graph](https://www.tensorflow.org/api_docs/python/tf/Graph)).
2. Cálculo de la **gráfica computacional** (utilizando una [tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session)).

## Gráfica computacional
Es una serie de **operaciones** de TensorFlow formando una gráfica. Esta está compuesta de objetos de dos tipos:
- **[Operaciones](https://www.tensorflow.org/api_docs/python/tf/Operation)**: Son los nodos de la gráfica. Representan cálculos a realizar sobre **tensores**. Producen a su vez nuevos **tensores**.
- **[Tensores](https://www.tensorflow.org/api_docs/python/tf/Tensor)**: Son las líneas de la gráfica. Outputs de las **operaciones**. Representan los valores que fluyen por la gráfica.

<div style="margin-left: 25px;margin-right: 150px;background-color: coral; padding: 10px">**Nota**: Los tensores no tienen valores, solo son llaves para acceder a elementos de la gráfica computacional.</div>

### Creando una gráfica computacional
Para crear una gráfica computacional debo crear operaciones que produzcan tensores. La operación más básica es [tf.constant](https://www.tensorflow.org/api_docs/python/tf/constant). Esta requiere que se le pase un valor. Cuando se compute esta operación, producirá un tensor con el valor que se pasó a la misma.

In [15]:
# Reseteamos la gráfica actual para eliminar los tensores creados anteriormente
tf.reset_default_graph()

# Tensor con shape=() y valor constante de 3
a = tf.constant(3.0)
b = tf.constant(4.0)
print(a)
print(b)

# Tensor attributes 
a.__dict__

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)


{'_consumers': [],
 '_dtype': tf.float32,
 '_handle_data': None,
 '_op': <tf.Operation 'Const' type=Const>,
 '_shape': TensorShape([]),
 '_value_index': 0}

Se puede observar que al imprimir las variables, no obtenemos los valores `3.0` y `4.0`. Esto es porque lo unico que se hizo hasta ahora fue construir la gráfica. Estos objetos **Tensor** solo representan los resultados de las operaciones que se van a ejecutar.

Al crear operaciones, estas se guardan en la gráfica con un nombre único, que es independiente al nombre de los objetos en Python (`a`, `b`).

Vamos a crear una nueva operación en la gráfica, que consuma los tensores producidos por las operaciones ya creadas.

In [16]:
simple_sum = a + b
print(simple_sum)

Tensor("add:0", shape=(), dtype=float32)


Creamos una operación que suma los tensores obtenidos de las operaciones tf.constant. Nuevamente vemos que el tensor no posee un valor, porque todavía no ejecutamos la gráfica. TensorFlow automáticamente añade la operación a la gráfica y relaciona tensores con operaciones.

Como se puede ver, los atributos del tensor `a` cambiaron:

In [17]:
# Tensor attributes 
a.__dict__

{'_consumers': [<tf.Operation 'add' type=Add>],
 '_dtype': tf.float32,
 '_handle_data': None,
 '_op': <tf.Operation 'Const' type=Const>,
 '_shape': TensorShape([]),
 '_value_index': 0}

### Visualizando la gráfica con TensorBoard

TensorBoard es una herramienta de TensorFlow con muchas utilidades. Una de ellas es visualizar la gráfica computacional con la que se creó. Para utilizar está funcionalidad, primero debemos crear y guardar un archivo con la información que necesita TensorBoard. Para esto debemos agregar en nuestro programa:

In [18]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

Estas lineas de código crean un archivo en la carpeta del programa actual. Para utilizar TensorBoard con dicho archivo debemos abrir una consola, navegar hasta la carpeta del archivo y activar el ambiente de trabajo del programa actual (que tiene instalado TensorFlow). Debemos ingresar el comando:
```bash
tensorboard --logdir .
```
Esto comenzará a ejecutar TensorBoard. Para visualizarlo debemos abrir en el explorador, la dirección en la cuál se está ejecutando, provista en la consola luego de ejecutar el comando (ej: `http://localhost:6006/`).

La gráfica que realizamos hasta el momento, en TensorBoard, se ve de esta forma:

![Simple sum graph](simple_sum.png "Simple sum graph")


### Dandole nombres a los tensores
Se observa que al crear los anteriores tensores, TensorFlow les asigna un nombre ("Const", "Const_1" y "add") que son los que usa para mostrar en la gráfica. Podemos darle nuestros propios nombres a los tensores utilizando el atributo `name` de la mayoria de las operaciones.

In [29]:
# Reseteamos la gráfica actual para eliminar los tensores creados anteriormente
tf.reset_default_graph()

a = tf.constant(3.0, dtype=tf.float32, name="input")
b = tf.constant(4.0, dtype=tf.float32, name="input")

# simple_sum = a + b
simple_sum = tf.add(a, b, name="simple_sum")
print(a)
print(b)
print(simple_sum)

Tensor("input:0", shape=(), dtype=float32)
Tensor("input_1:0", shape=(), dtype=float32)
Tensor("simple_sum:0", shape=(), dtype=float32)


In [30]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

Nueva gráfica de TensorBoard:

![Simple sum graph_names](simple_sum_named.png "Simple sum with names graph")

Otro punto a destacar con respecto a los nombres, es que TensorFlow nunca va a permitir dos tensores distitos con el mismo nombre. En el ejemplo `a` y `b` utilizaron el mismo nombre, pero TensorFlow le agregó un "_1" para distinguiro.

## Cálculo de la gráfica computacional
Para que los tensores nos devuelvan un valor, debemos ejecutar los cálculos de la gráfica utilizando un objeto [tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session) (sesión). Este objeto calcula el valor del tensor de salida de cualquier operación, o nodo, de la gráfica, y todos los anteriores. Para esto, utilizamos la función `run` de la sesión.

Vamos a obtener el valor de los tensores de nuestra geáfica:

In [31]:
# Utilización de tf.Session por convención
with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(simple_sum))

3.0
4.0
7.0


En la misma ejecución de [tf.Session.run](https://www.tensorflow.org/api_docs/python/tf/InteractiveSession#run), puedo obtener el valor de varios tensores de la gráfica:

In [32]:
with tf.Session() as sess:
    print(sess.run([a, b, simple_sum]))

[3.0, 4.0, 7.0]


Algunas operaciones de TensorFlow, devuelven operaciones en lugar de tensores. Cuando ejecutamos una operación de este tipo con **tf.Session.run**, esta nos devuelve `None`. Ejecutar una una de estas operaciones tiene la finalidad de realizar alguna acción, y no obtener un valor. Algunos ejemplos son las operaciones de inicialización y entrenamiento.

### Alimentación de datos a la gráfica
Para poder utilizar la gráfica con distintos valores de entrada, y obtener distintos valores de salida, debemos usar el parametro `feed_dict` de la función `tf.Session.run`. Este recibe un diccionario con el nombre del tensor al que le quiero asignar un valor, y el valor a asignar.

In [33]:
with tf.Session() as sess:
    print(sess.run(simple_sum, feed_dict={a: 1.0, b: 2.0}))

3.0


### Utilización de Placeholders
Las operaciones [tf.placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) sirven como punto de entrada de datos en una gráfica. El cálculo de una operación con `tf.Session.run` que dependa de un placeholder, requiere que se utilice el diccionario de alimentación con los valores que deben asumir los tensores generados por las operaciones de placeholders.`

In [34]:
# Reseteamos la gráfica actual para eliminar los tensores creados anteriormente
tf.reset_default_graph()

x = tf.placeholder(dtype=tf.float32, name="input_x")
y = tf.placeholder(dtype=tf.float32, name="input_y")
z = x + y

print(x)
print(y)
print(z)

Tensor("input_x:0", dtype=float32)
Tensor("input_y:0", dtype=float32)
Tensor("add:0", dtype=float32)


In [35]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

![Simple sum graph_placeholders](simple_sum_placeholders.png "Simple sum with names graph")

In [36]:
with tf.Session() as sess:
    print(sess.run(z, feed_dict={x: 2.0, y: 5.0}))

7.0


In [14]:
# Reseteamos la gráfica actual para eliminar los tensores creados anteriormente
tf.reset_default_graph()

x = tf.placeholder(dtype=tf.float32, name="x")
w = tf.Variable(tf.random_normal([1]), dtype=tf.float32, name="w")

y = x * w
w.__dict__

{'_caching_device': None,
 '_initial_value': <tf.Tensor 'random_normal:0' shape=(1,) dtype=float32>,
 '_initializer_op': <tf.Operation 'w/Assign' type=Assign>,
 '_save_slice_info': None,
 '_snapshot': <tf.Tensor 'w/read:0' shape=(1,) dtype=float32>,
 '_variable': <tf.Tensor 'w:0' shape=(1,) dtype=float32_ref>}

In [15]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

In [ ]:


with tf.Session() as sess:
    

## Referencias
 - https://www.tensorflow.org/programmers_guide/low_level_intro
